In [1]:
from collections import namedtuple
import csv
import math
import numpy as np
import matplotlib.pyplot as plt

import nbimporter
import GlassAngularProperty
import IncidentSolarRadiationOnGlazing
import MultipleReflectionOfSolarRadiation
import DivisionDiffuseRatio

Importing Jupyter notebook from GlassAngularProperty.ipynb
Importing Jupyter notebook from IncidentSolarRadiationOnGlazing.ipynb
Importing Jupyter notebook from MultipleReflectionOfSolarRadiation.ipynb
Importing Jupyter notebook from DivisionDiffuseRatio.ipynb


# 開口面透過日射量を計算するモジュール

## 1. Functions  

開口面日射量（直達・天空・地表面反射）とグレージング複合体の日射透過率から開口面透過日射量を計算する。

### 入力値  

$ I_{T,b} $ ：斜面（開口面）直達日射量 (W/m<sup>2</sup>)  
$ I_{T,d} $ ：斜面（開口面）天空日射量 (W/m<sup>2</sup>)  
$ I_{T,r} $ ：斜面（開口面）地表面反射日射量 (W/m<sup>2</sup>)  
$ \tau_{glz,b} $ ：直達日射に対する日射透過率  
$ \tau_{glz,d,l,m} $ ：天空上の微小要素からの入射日射に対する日射透過率  
$ \tau_{glz,r,l,m} $ ：地表面上の微小要素からの入射日射に対する日射透過率  
$ F_{d,l,m} $ ：天空日射量に対する天空上の微小要素からの天空日射量の割合  
$ F_{r,l,m} $ ：地表面反射日射量に対する地表面上の微小要素からの地表面反射日射量の割合  

### 出力値  

$ I_{\tau} $ ：開口面透過日射量 (W/m<sup>2</sup>)  

### 計算方法  

#### データ構造と値の範囲  

$ \tau_{glz,b} $・$ \tau_{glz,d} $・$ \tau_{glz,r} $ の組み合わせを入力条件として1つの変数（タプル）で表し"tta_tuple"とする。  

In [2]:
def tta_input():
    return namedtuple('tta_tuple','tta_taub tta_taud tta_taur')

#tta_taub：直達成分の日射透過率
#tta_taud：天空成分の日射透過率
#tta_taur：地表面反射成分の日射透過率

#### 透過日射量の計算  

透過日射量の計算方法を以下に示す。

$ \begin{align}I_{\tau} = I_{T,b}･\tau_{glz,b} + I_{T,d}･\tau_{glz,d} + I_{T,r}･\tau_{glz,r} \end{align}$ ････････････････････ (5.1.4.3-a)  

$ \begin{align}\tau_{glz,d} = \sum_{l=0}^{N_{d,lat}-1}\sum_{m=0}^{N_{d,lon}-1} F_{d,l,m}･\tau_{glz,d,l,m} \end{align}$ ････････････････････ (5.1.4.3-b)  

$ \begin{align}\tau_{glz,r} = \sum_{l=0}^{N_{r,lat}-1}\sum_{m=0}^{N_{r,lon}-1} F_{r,l,m}･\tau_{glz,r,l,m} \end{align}$ ････････････････････ (5.1.4.3-c)  

In [3]:
#開口面透過日射量
def transmit_sol(L, M):
    
    return L.ita_Itb * M.tta_taub + L.ita_Itd * M.tta_taud + L.ita_Itr * M.tta_taur

#直達日射での透過率
def actual_trans_beam(L, phi_b):
    
    r_taub = 0.
    ly_in_b = np.empty(len(L), dtype = tuple)
    for j in range(0, len(L)):
        ly_in_b[j] = MultipleReflectionOfSolarRadiation.layer_input() \
                     (GlassAngularProperty.glass_TRSf(L[j], phi_b), GlassAngularProperty.glass_TRSb(L[j], phi_b), \
                      GlassAngularProperty.glass_REFf(L[j], phi_b), GlassAngularProperty.glass_REFb(L[j], phi_b))
    
    r_taub = MultipleReflectionOfSolarRadiation.Sol_Trs_All(ly_in_b)
                
    return r_taub

#天空日射での透過率
def actual_trans_diffuse(L, Nd_lat, Nd_lon):

    F_d = np.empty((Nd_lat, Nd_lon))
    phi_d = np.empty((Nd_lat, Nd_lon))
    tau_d = np.empty((Nd_lat, Nd_lon))
    r_taud = 0.
    ly_in_d = np.empty(len(L), dtype = tuple)
    for l in range(0, Nd_lat):
        for m in range(0, Nd_lon):
            dd_in = DivisionDiffuseRatio.dd_input()(Nd_lat, Nd_lon, l, m)
            F_d[l][m] = DivisionDiffuseRatio.division_diffuse_ratio(dd_in)[0]   #割合
            phi_d[l][m] = DivisionDiffuseRatio.division_diffuse_ratio(dd_in)[3] #入射角
            
            for j in range(0, len(L)):
                ly_in_d[j] = MultipleReflectionOfSolarRadiation.layer_input() \
                             (GlassAngularProperty.glass_TRSf(L[j], phi_d[l][m]), GlassAngularProperty.glass_TRSb(L[j], phi_d[l][m]), \
                              GlassAngularProperty.glass_REFf(L[j], phi_d[l][m]), GlassAngularProperty.glass_REFb(L[j], phi_d[l][m]))

            tau_d[l][m] = MultipleReflectionOfSolarRadiation.Sol_Trs_All(ly_in_d)

            r_taud += F_d[l][m] * tau_d[l][m]
               
    return r_taud

#地表面反射日射での透過率
def actual_trans_reflect(L, Nr_lat, Nr_lon):

    F_r = np.empty((Nr_lat, Nr_lon))
    phi_r = np.empty((Nr_lat, Nr_lon))
    tau_r = np.empty((Nr_lat, Nr_lon))
    r_taur =0.
    ly_in_r = np.empty(len(L), dtype = tuple)  
    for l in range(0, Nr_lat):
        for m in range(0, Nr_lon):
            dr_in = DivisionDiffuseRatio.dr_input()(Nr_lat, Nr_lon, l, m)
            F_r[l][m] = DivisionDiffuseRatio.division_reflect_ratio(dr_in)[0]   #割合
            phi_r[l][m] = DivisionDiffuseRatio.division_reflect_ratio(dr_in)[3] #入射角

            for j in range(0, len(L)):
                ly_in_r[j] = MultipleReflectionOfSolarRadiation.layer_input() \
                             (GlassAngularProperty.glass_TRSf(L[j], phi_r[l][m]), GlassAngularProperty.glass_TRSb(L[j], phi_r[l][m]), \
                              GlassAngularProperty.glass_REFf(L[j], phi_r[l][m]), GlassAngularProperty.glass_REFb(L[j], phi_r[l][m]))

            tau_r[l][m] = MultipleReflectionOfSolarRadiation.Sol_Trs_All(ly_in_r)

            r_taur += F_r[l][m] * tau_r[l][m]
                
    return r_taur

#### Example

In [4]:
if __name__ == '__main__':

    #ガラス光学特性
    gl_in = [GlassAngularProperty.glass_input()(0.815, 0.815, 0.072, 0.072, 0, 0, 0),
             GlassAngularProperty.glass_input()(0.815, 0.815, 0.072, 0.072, 0, 0, 0),
             GlassAngularProperty.glass_input()(0.815, 0.815, 0.072, 0.072, 0, 0, 0)]

    #斜面日射量
    ita_in = IncidentSolarRadiationOnGlazing.ita_input()(2.6840261248 * 10. ** 6. / 3600., 
                                                         0.1181932153 * 10. ** 6. / 3600., 
                                                         0.094 * 10. ** 6. / 3600.)

    #直達日射の入射角
    phi_b = 31.6403837639

    #天空分割数
    Nd_lat = 90
    Nd_lon = 180

    #地表面反射分割数
    Nr_lat = 90
    Nr_lon = 180

    tau_b = actual_trans_beam(gl_in, phi_b)
    tau_d = actual_trans_diffuse(gl_in, Nd_lat, Nd_lon)
    tau_r = actual_trans_reflect(gl_in, Nr_lat, Nr_lon)

    tta_in = tta_input()(tau_b, tau_d, tau_r)
    I_tau = transmit_sol(ita_in, tta_in)                  #斜面（開口面）透過日射量

    print (I_tau)

418.40754179495804
